## Análise de homologias por BLAST

In [1]:
from Bio import SeqIO
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

In [2]:
files = [("rrnB","dna","blastn","refseq_rna"),
         ("ampC","dna","blastn","nt"),
         ("ampC","prot","blastp","swissprot")]

In [4]:
for file in files:
    print(f"----> E. coli {file[0]} ({file[1]}) BLAST <----\n")
    # --> ler o ficheiro fasta e correr o blast <--
    record = SeqIO.read(f"{file[0]}_{file[1]}.fasta","fasta")
    handle = NCBIWWW.qblast(file[2],file[3],record.seq,hitlist_size=100,
                            entrez_query="all [filter] NOT(txid561[ORGN] OR txid29278[ORGN] "
                            "OR txid32630[ORGN] OR txid77133[ORGN] OR txid2608721[ORGN])")
    # --> guardar os dados recolhidos num ficheiro xml <--
    with open(f"blast_{file[0]}_{file[1]}.xml","w") as out_handle:
        out_handle.write(handle.read())
    handle.close()
    # --> procurar alinhamentos com e-value < 0.0001 e guardar as respetivas sequências<--
    handle = open(f"blast_{file[0]}_{file[1]}.xml")
    blast_records = NCBIXML.parse(handle)
    output = open(f"homol_{file[0]}_{file[1]}.fasta","a")
    output.write(f">{record.id}\n{record.seq}\n\n") # guardar a sequência de E. coli
    e_value_threshold = 0.0001
    counter = 1
    titles_list = [] # "keep track" dos alignment.titles de modo a evitar nomes duplicados
    for blast_record in blast_records:
        if blast_record:
            for alignment in blast_record.alignments:
                for hsp in alignment.hsps:
                    if hsp.expect < e_value_threshold:
                        if alignment.title not in titles_list:
                            titles_list.append(alignment.title)
                            print (f"**** Alignment {counter} ****")
                            counter += 1
                            print ("sequence:", alignment.title)
                            print ("length:", alignment.length)
                            print ("e-value:", hsp.expect)
                            print(hsp.query[0:60] + "...")
                            print(hsp.match[0:60] + "...")
                            print(hsp.sbjct[0:60] + "...")
                            print("")
                            output.write(f">{alignment.title}\n{hsp.sbjct}\n\n")
    handle.close()
    output.close()
    print("\n")

----> E. coli rrnB (dna) BLAST <----

**** Alignment 1 ****
sequence: gi|559795236|ref|NR_104826.1| Shigella sonnei strain CECT 4887 16S ribosomal RNA, partial sequence
length: 1530
e-value: 0.0
GGCTCAGATTGAACGCTGGCGGCAGGCCTAACACATGCAAGTCGAACGGTAACAGGAAGA...
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||  ...
GGCTCAGATTGAACGCTGGCGGCAGGCCTAACACATGCAAGTCGAACGGTAACAGGAAAC...

**** Alignment 2 ****
sequence: gi|559795308|ref|NR_104901.1| Shigella boydii strain P288 16S ribosomal RNA, partial sequence
length: 1515
e-value: 0.0
ATTGAACGCTGGCGGCAGGCCTAACACATGCAAGTCGAACGGTAACAGGAAGAAGCTTGC...
|||||||||||||||||||||||||||||||||||||||||||||||||||| |||||||...
ATTGAACGCTGGCGGCAGGCCTAACACATGCAAGTCGAACGGTAACAGGAAGCAGCTTGC...

**** Alignment 3 ****
sequence: gi|219846739|ref|NR_026331.1| Shigella flexneri strain ATCC 29903 16S ribosomal RNA, partial sequence
length: 1488
e-value: 0.0
TGGCTCAGATTGAACGCTGGCGGCAGGCCTAACACATGCAAGTCGAACGGTAACAGGAAG...
|||||||||||||||||||||||||||||||||||||||||||